<a href="https://colab.research.google.com/github/lmaura37/SIS-420_M.C.L/blob/main/EXA2maura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/024_mlp_clasificacion/mlp_clasificacion.ipynb)

# El Perceptrón Multicapa - Clasificación

## El Perceptrón Multicapa

En el [post](https://sensioai.com/blog/023_mlp_backprop) anterior hemos introducido el modelo de `Perceptrón Multicapa`, o MLP, la arquitectura de red neuronal más básica basada en el [Perceptrón](https://sensioai.com/blog/012_perceptron1). Hemos visto cómo calcular la salida de un MLP de dos capas a partir de unas entradas y cómo encontrar los pesos óptimos para una tarea de regresión. En este post vamos a mejorar la implementación de nuestro MLP de dos capas para que sea capaz también de llevar a cabo tareas de clasificación.

![](https://www.researchgate.net/profile/Mohamed_Zahran6/publication/303875065/figure/fig4/AS:371118507610123@1465492955561/A-hypothetical-example-of-Multilayer-Perceptron-Network.png)

## Implementación

La mayoría del código que utilizaremos fue desarrollado para el `Perceptrón` y lo puedes encontrar en este [post](https://sensioai.com/blog/018_perceptron_final). Lo único que cambiaremos es la lógica del modelo, el resto de funcionalidad (funciones de pérdida, funciones de activación, etc, siguen siendo exactamente igual).

### Funciones de activación

Para la capa oculta de nuestro MLP utilizaremos una función de activación de tipo `relu`, de la cual necesitaremos su derivada.

In [1]:
def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

En cuanto a las funciones de activación que utilizaremos a la salida del MLP, éstas son las que hemos introducido en posts anteriores:

- Lineal: usada para regresión (junto a la función de pérdida MSE).
- Sigmoid: usada para clasificación binaria (junto a la función de pérdida BCE).
- Softmax: usada para clasificación multiclase (junto a la función de pérdida crossentropy, CE).

In [2]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

### Funciones de pérdida

Como acabamos de comentar en la sección anterior, estas son las funciones de pérdida que hemos visto hasta ahora para las diferentes tareas.

In [3]:
# Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)

# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

Y sus derivadas

In [4]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1    
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

### Implementación MLP

Ahora que ya tenemos definidas las diferentes funciones de activación y de pérdida que necesitamos, vamos a implementar nuestro MLP de dos capas capaz de llevar a cabo tanto tareas de regresión como de clasificación. Del mismo modo que ya hicimos con el `Perceptrón`, definiremos una clase base que servirá para la implementación de las clases particulares para cada caso.

In [5]:
# clase base MLP 

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2 
    return self.activation(y_hat)
    
  def fit(self, X, Y, epochs = 10000, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):     
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size] 
            # salida del perceptrón
            y_pred = self(x) 
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)        
            # Backprop 
            dldy = self.grad_loss(y, y_pred) 
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)      
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [6]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)

# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

Vamos a probar ahora nuestra implementación para diferentes ejemplos.

## Regrecion

In [7]:
# utilizado para manejos de directorios y rutas
import os

# Computacion vectorial y cientifica para python
import numpy as np
import pandas as pd
import sys
# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # Necesario para graficar superficies 3D
import matplotlib.pyplot as plt
# llama a matplotlib a embeber graficas dentro de los cuadernillos
%matplotlib inline

In [8]:
data = np.loadtxt('/content/fiderating_dataset_examen.txt', delimiter=';')
X, y = data[:,: 7], data[:,7]
print(X.shape, y.shape)

(8100, 7) (8100,)


In [9]:
y

array([2064., 2084., 2072., ..., 2689., 2689., 2686.])

In [10]:
#normalizar el X
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
X_norm = (X - X_mean) / X_std
print(X_norm)
#normalizando Y
y_mean1, y_std1 = y.mean(axis=0), y.std(axis=0)
y_norm = (y - y_mean1) / y_std1


[[ 1.90750039 -1.68005542 -2.67299351 ... -0.77693912 -0.22054608
   1.19717421]
 [ 1.88702439 -1.65958617 -2.62576054 ... -0.77693912  1.05496945
   0.62197038]
 [ 1.8665484  -1.63911692 -2.57800853 ... -0.77693912 -0.9858554
   1.74731001]
 ...
 [-1.63484613  1.94300189  1.38748491 ... -0.90817975 -0.73075229
   0.17101701]
 [-1.65532212  1.96347114  1.40305622 ... -0.90817975 -0.22054608
  -0.90437697]
 [-1.67579811  1.98394039  1.41914658 ... -0.90817975 -0.41187341
  -0.68905795]]


Sin embargo, nuestro nuevo modelo, el MLP, es capaz de solventar este problema.

In [11]:
##X_train1, X_test1, y_train1, y_test1 = X_norm[:7000], X_norm[7000:], y_norm[:7000], y_norm[7000:]

In [12]:
model = MLPRegression(D_in=7, H=20, D_out=1)
epochs, lr = 500, 0.001
model.fit(X_norm.reshape(len(X_norm),7), y_norm, epochs, lr, batch_size=1, log_each=100)

Epoch: 100/500, Loss: 0.06461
Epoch: 200/500, Loss: 0.04723
Epoch: 300/500, Loss: 0.03946
Epoch: 400/500, Loss: 0.03537
Epoch: 500/500, Loss: 0.03289


In [13]:
##Predicion 1 tiene que dar
x_new = [164,12,37987,45,1503014,34.0,0.018115942028985508]
x_new = (x_new-X_mean)/X_std
y_predd = model(x_new)
print("prediccion")
print(y_predd * y_mean1)

prediccion
[-2597.77272116]


In [14]:
x_new1 = [127,38,40909,6,8603677,0.0,0.002255639097744361] 
#x_new = (x_new-X_mean)/X_std
y_predd = model(x_new1)
print("prediccion ")
print(y_predd * y_mean1 )

prediccion 
[-3.6994954e+09]


In [15]:
x_new = [130,35,40725,15,8603677,10.0,0.005649717514124294] 
x_new = (x_new-X_mean)/X_std
y_predd = model(x_new)
print("prediccion ")
print(y_predd * y_mean1)

prediccion 
[158.8562869]


In [16]:
x_new = [133,32,40544,18,8603677,24.0,0.00684931506849315]
x_new = (x_new-X_mean)/X_std
y_predd = model(x_new)
print("prediccion ")
print(y_predd * y_mean1)

prediccion 
[-143.58179281]


In [17]:
x_new = [149,27,39356,25,1503014,4.0,0.009211495946941784]
x_new = (x_new-X_mean)/X_std
y_predd = model(x_new)
print("prediccion ")
print(y_predd * y_mean1)

prediccion 
[1103.39640532]


In [18]:
x_new = [153,23,38991,46,1503014,23.0,0.01704966641957005]
x_new = (x_new-X_mean)/X_std
y_predd = model(x_new)
print("prediccion ")
print(y_predd * y_mean1)

prediccion 
[802.13172874]


In [19]:
x_new = [155,21,38808,13,1503014,21.0,0.0049130763416477706]
x_new = (x_new-X_mean)/X_std
y_predd = model(x_new)
print("prediccion ")
print(y_predd * y_mean1)

prediccion 
[92.5393279]


Como puedes observar, el MLP es capaz de resolver el problema del `Perceptrón`, y es que cuantas más capas y neuronas por capas usemos, mayor capacidad de representación tendrá el modelo (las redes neuronales más grandes a día de hoy tienen varios Billones de conexiones).

## Clasificación Multiclase

Por último vamos a ver cómo aplicar nuestro modelo para clasificación en multiples clases.

In [ ]:
# importa numpy 
# leer tu data set
# normalizar
# llamar a la fun de MLPClassification

#Prediciones

In [ ]:
model = MLPClassification(D_in=2, H=10, D_out=3)
epochs, lr = 50, 0.2
model.fit(X_norm, y, epochs, lr, batch_size=10, log_each=10)

De nuevo, nuestro MLP es capaz de separar las tres clases en el dataset. En posts anteriores hemos trabajado también con el dataset MNIST para clasificación de imágenes en diez clases distintas. ¿Te ves capaz de utilizar nuestro MLP para resolver ese problema?

## Resumen

En este post hemos visto como implementar un `Perceptrón Multicapa` en `Python` para tareas de regresión y clasificación. Como ya hicimos anteriormente para el caso del `Perceptrón` hemos validado nuestra implementación con el dataset de clasificación de flores *Iris*, tanto para clasificación binaria como multiclase. Sin embargo, nuestra implementación está muy limitada. ¿Qué pasa si queremos usar un MLP de más de dos capas?, ¿y si queremos usar una función de activación diferente a la `relu` en la capa oculta?, ¿podríamos utilizar un algoritmo de optimizaciñon diferente al `descenso por gradiente`? Para poder hacer todo esto necesitamos un `framework` más flexible, similar a lo que nos ofrecen `Pytorch` y `Tensorflow`. En el siguiente post desarrollaremos nuestro propio framework de MLP para que sea más flexible y que también nos servirá para entender cómo funcionan el resto de frameworks de `redes neuronales` por dentro.